In [15]:
import biome.text
import pandas as pd 
from langdetect import detect

In [16]:
model = biome.text.Pipeline.from_pretrained("/mnt/s/stiven/proyects/Temis-main (1)/Temis-main/models/fine-tuned_model.tar.gz")

In [17]:
def convert_dict(d:str):
	lists = tuple(d.values())
	l = []
	for tupla in zip(lists[0][:4], lists[1][:4]):
		l.append((tupla[0], tupla[1]*100))

	return l

In [18]:
def get_misoginy(lyric):
    ent_m = 0
    lyric_list = lyric.split('<>')
    lyric_list.remove('')
    for e in lyric_list:
        try:
            ls = convert_dict(model.predict(e))
            a = ls[0][0]
            b = ls[0][1]

            if a == 'active' or a == 'pasive':
                a = ls[1][0]
                b = ls[0][1]
            if b > 50:
                ent_m += 1
        except:
            continue
    return (e, (a, b), ent_m)

In [19]:
tango = pd.read_csv('/mnt/s/stiven/proyects/final_cuanti/songs_by_genre/tango.tsv', sep='|', header=0,
                 names = ['id', 'song_name', 'artist', 'lyric', 'lyric_by_e'])
# #tango.lyric.value_counts()
# # elmininar valores nulos
tango = tango.dropna(axis = 0, how ='any') 
# identificar el idioma
tango['lang'] = tango['lyric'].apply(detect)
# crear una columna con el tamaña de la letra en caracteres
tango["lyric_length"]= tango["lyric"].str.len()
# elimina canciones duplicadas
tango = tango.drop_duplicates(['song_name'])
# elminar canciones que no esten en español
indexLang = tango[tango['lang']!='es'].index
tango.drop(indexLang, inplace=True)
tango.head()

,id,song_name,artist,lyric,lyric_by_e,lang,lyric_length
0,1,Volver,Carlos Gardel,Yo adivino el parpadeo De las luces que a lo l...,Yo adivino el parpadeo De las luces que a lo l...,es,1223
1,2,Nostalgias,Carlos Gardel,Quiero emborrachar mi corazón Para apagar un l...,Quiero emborrachar mi corazón Para apagar un l...,es,1323
2,3,Cambalache,Carlos Gardel,"El mundo fue y será una porquería, ya lo sé En...","El mundo fue y será una porquería, ya lo sé En...",es,1404
3,4,El Día Que Me Quieras,Carlos Gardel,Acaricia mi ensueño el suave murmullo de tu su...,Acaricia mi ensueño el suave murmullo de tu su...,es,874
4,5,Mano A Mano,Carlos Gardel,"Rechiflado en mi tristeza, hoy te evoco y veo ...","Rechiflado en mi tristeza, hoy te evoco y veo ...",es,1483


In [20]:
# evaluar misoginia por canción
tango['eval'] = tango['lyric'].apply(model.predict)

,id,song_name,artist,lyric,lyric_by_e,lang,lyric_length,eval
0,1,Volver,Carlos Gardel,Yo adivino el parpadeo De las luces que a lo l...,Yo adivino el parpadeo De las luces que a lo l...,es,1223,"{'labels': ['stereotype', 'active', 'passive',..."
1,2,Nostalgias,Carlos Gardel,Quiero emborrachar mi corazón Para apagar un l...,Quiero emborrachar mi corazón Para apagar un l...,es,1323,"{'labels': ['stereotype', 'active', 'sexual_ha..."
2,3,Cambalache,Carlos Gardel,"El mundo fue y será una porquería, ya lo sé En...","El mundo fue y será una porquería, ya lo sé En...",es,1404,"{'labels': ['passive', 'stereotype', 'active',..."
3,4,El Día Que Me Quieras,Carlos Gardel,Acaricia mi ensueño el suave murmullo de tu su...,Acaricia mi ensueño el suave murmullo de tu su...,es,874,"{'labels': ['active', 'sexual_harassment', 'st..."
4,5,Mano A Mano,Carlos Gardel,"Rechiflado en mi tristeza, hoy te evoco y veo ...","Rechiflado en mi tristeza, hoy te evoco y veo ...",es,1483,"{'labels': ['active', 'stereotype', 'sexual_ha..."


In [21]:
tango['eval'] = tango['eval'].apply(convert_dict)
tango.head()

,id,song_name,artist,lyric,lyric_by_e,lang,lyric_length,eval
0,1,Volver,Carlos Gardel,Yo adivino el parpadeo De las luces que a lo l...,Yo adivino el parpadeo De las luces que a lo l...,es,1223,"[(stereotype, 4.225970804691315), (active, 2.2..."
1,2,Nostalgias,Carlos Gardel,Quiero emborrachar mi corazón Para apagar un l...,Quiero emborrachar mi corazón Para apagar un l...,es,1323,"[(stereotype, 3.566819801926613), (active, 3.3..."
2,3,Cambalache,Carlos Gardel,"El mundo fue y será una porquería, ya lo sé En...","El mundo fue y será una porquería, ya lo sé En...",es,1404,"[(passive, 2.739008329808712), (stereotype, 2...."
3,4,El Día Que Me Quieras,Carlos Gardel,Acaricia mi ensueño el suave murmullo de tu su...,Acaricia mi ensueño el suave murmullo de tu su...,es,874,"[(active, 82.59331583976746), (sexual_harassme..."
4,5,Mano A Mano,Carlos Gardel,"Rechiflado en mi tristeza, hoy te evoco y veo ...","Rechiflado en mi tristeza, hoy te evoco y veo ...",es,1483,"[(active, 3.1844981014728546), (stereotype, 2...."


In [22]:
# evaluar misoginia por estrofa
tango['misogy_estr'] = tango['lyric_by_e'].apply(get_misoginy)

In [23]:
tango.head()

,id,song_name,artist,lyric,lyric_by_e,lang,lyric_length,eval,misogy_estr
0,1,Volver,Carlos Gardel,Yo adivino el parpadeo De las luces que a lo l...,Yo adivino el parpadeo De las luces que a lo l...,es,1223,"[(stereotype, 4.225970804691315), (active, 2.2...",(Volver con la frente marchita Las nieves del ...
1,2,Nostalgias,Carlos Gardel,Quiero emborrachar mi corazón Para apagar un l...,Quiero emborrachar mi corazón Para apagar un l...,es,1323,"[(stereotype, 3.566819801926613), (active, 3.3...",(Desde mi triste soledad Veré caer las rosas m...
2,3,Cambalache,Carlos Gardel,"El mundo fue y será una porquería, ya lo sé En...","El mundo fue y será una porquería, ya lo sé En...",es,1404,"[(passive, 2.739008329808712), (stereotype, 2....","(Siglo veinte, cambalache, problemático y febr..."
3,4,El Día Que Me Quieras,Carlos Gardel,Acaricia mi ensueño el suave murmullo de tu su...,Acaricia mi ensueño el suave murmullo de tu su...,es,874,"[(active, 82.59331583976746), (sexual_harassme...",(El día que me quieras endulzará sus cuerdas e...
4,5,Mano A Mano,Carlos Gardel,"Rechiflado en mi tristeza, hoy te evoco y veo ...","Rechiflado en mi tristeza, hoy te evoco y veo ...",es,1483,"[(active, 3.1844981014728546), (stereotype, 2....",(Y mañana cuando seas descolado mueble viejo Y...


In [24]:
tango.to_csv('./final_data/tango_with_e.csv', sep='|',  index=False)